# High-Fidelity Generative Image Compression

This colab can be used to compress images using HiFiC. This can also be achieved
by running `tfci.py`, as [explained in the README](https://github.com/tensorflow/compression/tree/master/models/hific#running-models-trained-by-us-locally).

Please visit [hific.github.io](https://hific.github.io) for more information.

## Setup Colab

In [ ]:
# Installs the latest version of TFC compatible with the installed TF version.
!pip install tensorflow-compression~=$(pip show tensorflow | perl -p -0777 -e 's/.*Version: (\d+\.\d+).*/\1.0/sg')

# Downloads the 'models' directory from Github.
![[ -e /tfc ]] || git clone https://github.com/tensorflow/compression /tfc
%cd /tfc/models

# Checks if tfci.py is available.
import tfci


## Enabling GPU

GPU should be enabled for this colab. If the next cell prints a warning, do the following:
- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down


In [ ]:
import tensorflow as tf

if not tf.config.list_physical_devices('GPU'):
  print('WARNING: No GPU found. Might be slow!')
else:
  print('Found GPU.')

## Imports and Definitions

In [ ]:
import os
import zipfile
from google.colab import files
import collections
from PIL import Image
from IPython.display import Image as DisplayImage
from IPython.display import Javascript
from IPython.core.display import display, HTML
import tfci
import urllib.request

tf.get_logger().setLevel('WARN')  # Only show Warnings

FILES_DIR = '/content/files'
OUT_DIR = '/content/out'
DEFAULT_IMAGE_URL = ('https://storage.googleapis.com/hific/clic2020/'
                     'images/originals/ad249bba099568403dc6b97bc37f8d74.png')

os.makedirs(FILES_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

File = collections.namedtuple('File', ['full_path', 'num_bytes', 'bpp'])

def print_html(html):
  display(HTML(html + '<br/>'))

def make_cell_large():
  display(Javascript(
      '''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

def get_default_image(output_dir):
  output_path = os.path.join(output_dir, os.path.basename(DEFAULT_IMAGE_URL))
  print('Downloading', DEFAULT_IMAGE_URL, '\n->', output_path)
  urllib.request.urlretrieve(DEFAULT_IMAGE_URL, output_path)


## Load files

In [ ]:
#@title Setup { vertical-output: false, run: "auto", display-mode: "form" }
#@markdown #### Custom Images
#@markdown Tick the following if you want to upload your own images to compress.
#@markdown Otherwise, a default image will be used.
#@markdown
#@markdown **Note**: We support JPG and PNG (without alpha channels).
#@markdown

upload_custom_images = False #@param {type:"boolean", label:"HI"}

if upload_custom_images:
  uploaded = files.upload()
  for name, content in uploaded.items():
    with open(os.path.join(FILES_DIR, name), 'wb') as fout:
      print('Writing', name, '...')
      fout.write(content)

#@markdown #### Select a model
#@markdown Different models target different bitrates.

model = 'hific-lo' #@param ["hific-lo", "hific-mi", "hific-hi"]


In [ ]:
if 'upload_custom_images' not in locals():
  print('ERROR: Please run the previous cell!')
  # Setting defaults anyway.
  upload_custom_images = False
  model = 'hific-lo'

In [ ]:
all_files = os.listdir(FILES_DIR)
if not upload_custom_images or not all_files:
  print('Downloading default image...')
  get_default_image(FILES_DIR)
  print()

all_files = os.listdir(FILES_DIR)
print(f'Got the following files ({len(all_files)}):')

for file_name in all_files:
  img = Image.open(os.path.join(FILES_DIR, file_name))
  w, h = img.size
  img = img.resize((w // 15, h // 15))
  print('- ' + file_name + ':')
  display(img)

# Compress images

In [ ]:
SUPPORTED_EXT = {'.png', '.jpg'}

all_files = os.listdir(FILES_DIR)
if not all_files:
  raise ValueError("Please upload images!")

def get_bpp(image_dimensions, num_bytes):
  w, h = image_dimensions
  return num_bytes * 8 / (w * h)

def has_alpha(img_p):
  im = Image.open(img_p)
  return im.mode == 'RGBA'

all_outputs = []
for file_name in all_files:
  if os.path.isdir(file_name):
    continue
  if not any(file_name.endswith(ext) for ext in SUPPORTED_EXT):
    print('Skipping', file_name, '...')
    continue
  full_path = os.path.join(FILES_DIR, file_name)
  if has_alpha(full_path):
    print('Skipping because of alpha channel:', file_name)
    continue
  file_name, _ = os.path.splitext(file_name)

  compressed_path = os.path.join(OUT_DIR, f'{file_name}_{model}.tfci')
  output_path = os.path.join(OUT_DIR, f'{file_name}_{model}.png')

  if os.path.isfile(output_path):
    print('Exists already:', output_path)
    num_bytes = os.path.getsize(compressed_path)
    all_outputs.append(
      File(output_path, num_bytes,
           get_bpp(Image.open(full_path).size, num_bytes)))
    continue

  print('Compressing', file_name, 'with', model, '...')
  tfci.compress(model, full_path, compressed_path)
  num_bytes = os.path.getsize(compressed_path)
  print(f'Compressed to {num_bytes} bytes.')

  print('Decompressing...')
  tfci.decompress(compressed_path, output_path)

  all_outputs.append(
      File(output_path, num_bytes,
           get_bpp(Image.open(full_path).size, num_bytes)))

print('All done!')

# Show output

In [ ]:
make_cell_large()  # Larger output window.

for file in all_outputs:
  print_html('<hr/>')
  print(f'Showing {file.full_path} | {file.num_bytes//1000}kB | {file.bpp:.4f}bpp')
  display(Image.open(file.full_path))
  print_html('<hr/>')

### Download all compressed images.

To download all images, run the following cell.

You can also use the _Files_ tab on the left to manually select images.

---

#### **Note**: the images are saved as PNGs and thus very large. The bitrate used by HiFiC is given in the name.

In [ ]:
ZIP = '/content/images.zip'

with zipfile.ZipFile(ZIP, 'w') as zf:
  for f in all_outputs:
    path_with_bpp = f.full_path.replace('.png', f'-{f.bpp:.3f}bpp.png')
    zf.write(f.full_path, os.path.basename(path_with_bpp))

files.download(ZIP)

In [1]:
!pip install -U huggingface_hub

In [2]:
!huggingface-cli download --token hf_** --repo-type dataset --resume-download crag-mm-2025/web-search-index-public --local-dir web-search-index-public

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 8 files:   0% 0/8 [00:00<?, ?it/s]Downloading '14dc4182-0002-4ca0-a226-d39043b81948/data_level0.bin' to 'web-search-index-public/.cache/huggingface/download/14dc4182-0002-4ca0-a226-d39043b81948/fbme-ciW2w7sDkVUQUA_cJWEH_M=.669a8bdae46737926fdc577ca7a4a8d7aba269f29fa00cf6ff1365e75f933b86.incomplete'

data_level0.bin:   0% 0.00/68.7M [00:00<?, ?B/s]Downloading '.gitattributes' to 'web-search-index-public/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.5bcf35802afab3d81672915a9fbaea4d0fad265a.incomplete'


index_metadata.pickle:   0% 0.00/1.09M [00:00<?, ?B/s]


link_lists.bin:   0% 0.00/357k [00:00<?, ?B/s]Downloading 'chroma.sqlite3' to 'web-search-index-public/.cache/huggingface/download/ti

In [1]:
!zip -r web-search-index-public.zip web-search-index-public

  adding: web-search-index-public/ (stored 0%)
  adding: web-search-index-public/.gitattributes (deflated 87%)
  adding: web-search-index-public/14dc4182-0002-4ca0-a226-d39043b81948/ (stored 0%)
  adding: web-search-index-public/14dc4182-0002-4ca0-a226-d39043b81948/index_metadata.pickle (deflated 55%)
  adding: web-search-index-public/14dc4182-0002-4ca0-a226-d39043b81948/header.bin (deflated 53%)
  adding: web-search-index-public/14dc4182-0002-4ca0-a226-d39043b81948/length.bin (deflated 85%)
  adding: web-search-index-public/14dc4182-0002-4ca0-a226-d39043b81948/link_lists.bin (deflated 72%)
  adding: web-search-index-public/14dc4182-0002-4ca0-a226-d39043b81948/data_level0.bin (deflated 12%)
  adding: web-search-index-public/chroma.sqlite3 (deflated 90%)
  adding: web-search-index-public/.cache/ (stored 0%)
  adding: web-search-index-public/.cache/huggingface/ (stored 0%)
  adding: web-search-index-public/.cache/huggingface/download/ (stored 0%)
  adding: web-search-index-public/.cache/

In [2]:
!huggingface-cli download --token hf_** --repo-type dataset --resume-download crag-mm-2025/image-search-index-public --local-dir image-search-index-public
!zip -r image-search-index-public.zip image-search-index-public

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 18 files:   0% 0/18 [00:00<?, ?it/s]Downloading '18571a51-4b0f-4c80-9b4d-f8a3f6789ca7/length.bin' to 'image-search-index-public/.cache/huggingface/download/18571a51-4b0f-4c80-9b4d-f8a3f6789ca7/k9LxY-gHgRP0VyiHwySuuheFRCQ=.003aad6a43292a671a716a61a43d6ab8290ee1e87d851e190cec6f138bfae845.incomplete'

header.bin: 100% 100/100 [00:00<00:00, 196kB/s]

Download complete. Moving file to image-search-index-public/452dead5-af12-4431-ae8e-cd40401df854/header.bin
length.bin:   0% 0.00/40.0k [00:00<?, ?B/s]

length.bin: 100% 40.0k/40.0k [00:00<00:00, 49.2MB/s]
Download complete. Moving file to image-search-index-public/18571a51-4b0f-4c80-9b4d-f8a3f6789ca7/length.bin

link_lists.bin:   0% 0.00/89.2k [00:00<?, ?B

In [7]:
!huggingface-cli download --token hf_** --repo-type dataset --resume-download crag-mm-2025/crag-mm-single-turn-public --local-dir crag-mm-single-turn-public
!zip -r crag-mm-single-turn-public.zip crag-mm-single-turn-public

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading 'data/sample-00000-of-00001.parquet' to 'crag-mm-single-turn-public/.cache/huggingface/download/data/yHQzVlRutg2_NlxjG48qCXvKFOc=.1b1ac28f2be448cae22d9573c52a4589d4a3cf1da42588063844de7fd5d8892d.incomplete'

sample-00000-of-00001.parquet:   0% 0.00/24.8M [00:00<?, ?B/s]
sample-00000-of-00001.parquet: 100% 24.8M/24.8M [00:00<00:00, 123MB/s]
Download complete. Moving file to crag-mm-single-turn-public/data/sample-00000-of-00001.parquet

README.md: 100% 4.84k/4.84k [00:00<00:00, 31.2MB/s]
Download complete. Moving file to crag-mm-single-turn-public/README.md

.gitattributes: 100% 2.46k/2.46k [00:00<00:00, 15.8MB/s]
Download complete. Moving file to crag-mm-

In [4]:
!huggingface-cli download --token hf_** --repo-type dataset --resume-download crag-mm-2025/crag-mm-multi-turn-public --local-dir crag-mm-multi-turn-public
!zip -r crag-mm-multi-turn-public.zip crag-mm-multi-turn-public

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 3 files:   0% 0/3 [00:00<?, ?it/s]Downloading 'data/sample-00000-of-00001.parquet' to 'crag-mm-multi-turn-public/.cache/huggingface/download/data/yHQzVlRutg2_NlxjG48qCXvKFOc=.98263e2165f80e2d5e4a4ca036675a77b53ff47ab1e6857af873f03cb528d9fa.incomplete'

sample-00000-of-00001.parquet:   0% 0.00/5.29M [00:00<?, ?B/s]

.gitattributes: 100% 2.46k/2.46k [00:00<00:00, 21.2MB/s]
Download complete. Moving file to crag-mm-multi-turn-public/.gitattributes
Fetching 3 files:  33% 1/3 [00:00<00:01,  1.56it/s]

README.md: 100% 4.63k/4.63k [00:00<00:00, 35.3MB/s]
Download complete. Moving file to crag-mm-multi-turn-public/README.md

sample-00000-of-00001.parquet: 100% 5.29M/5.29M [00:00<00:00, 7.90MB/s]
Download co

In [5]:
# !rm -rf crag-mm-multi-turn-public
# !rm -rf crag-mm-single-turn-public
# !rm -rf image-search-index-public
# !rm -rf web-search-index-public
# !rm -rf web-search-index-public*

In [6]:
# !huggingface-cli download --token hf_** --repo-type model --resume-download sentence-transformers/all-MiniLM-L6-v2 --local-dir all-MiniLM-L6-v2
# !zip -r all-MiniLM-L6-v2.zip all-MiniLM-L6-v2

!huggingface-cli download --token hf_** --repo-type model --resume-download openai/clip-vit-base-patch16 --local-dir clip-vit-base-patch16
!zip -r clip-vit-base-patch16.zip clip-vit-base-patch16






/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 11 files: 100% 11/11 [00:00<00:00, 4362.46it/s]
/content/clip-vit-base-patch16
updating: clip-vit-base-patch16/ (stored 0%)
updating: clip-vit-base-patch16/tokenizer_config.json (deflated 70%)
updating: clip-vit-base-patch16/.gitattributes (deflated 86%)
updating: clip-vit-base-patch16/pytorch_model.bin (deflated 35%)
updating: clip-vit-base-patch16/special_tokens_map.json (deflated 71%)
updating: clip-vit-base-patch16/flax_model.msgpack (deflated 35%)
updating: clip-vit-base-patch16/config.json (deflated 78%)
updating: clip-vit-base-patch16/preprocessor_config.json (deflated 42%)
updating: clip-vit-base-patch16/.cache/ (stored 0%)
updating: clip-vit-base-patch16/.cache/huggingface/ (stored 0%)
upda

In [ ]:
!rm -rf clip-vit*
!rm -rf all-MiniLM*